In [ ]:
# Assign an RPID to each parcel in our geo data
fay['RPID'] = fay['URL'].apply(lambda row: row[-5:])

In [65]:
s = requests.Session()
URL = "https://www.actdatascout.com/RealProperty/Arkansas/Washington"
for i, row in fay.iterrows():
    try:
        print(i)
        RPID = row['RPID']
        searchURL = "https://www.actdatascout.com/RealProperty/ParcelView?countyIdYearRpid=51432021" + str(RPID)
        s.get(URL)

        cookies = {'__RequestVerificationToken': s.cookies['__RequestVerificationToken'], 'ASP.NET_SessionId': s.cookies['ASP.NET_SessionId']}
        res = s.get(searchURL, headers={'Referer': 'https://www.actdatascout.com/RealProperty/Index'}, cookies=cookies)

        datalist = pd.read_html(res.content, header=0)
        ds = pd.Series()

        landValue = 0
        buildingValue = 0
        totalValue = 0
        val2019 = 0
        val2020 = 0

        for frame in datalist:
            if 'EstimatedMarket Value' in frame.columns:
                landValue = frame['EstimatedMarket Value'][0]
                buildingValue = frame['EstimatedMarket Value'][1]
                totalValue = frame['EstimatedMarket Value'][2]
            if 'Tax Year' in frame.columns:
                # get 2020 and 2019 values
                val2019 = frame['Total Value'][2]
                val2020 = frame['Total Value'][1]

        row['landValue'] = landValue
        row['buildingValue'] = buildingValue
        row['totalValue'] = totalValue
        row['value2019'] = val2019
        row['value2020'] = val2020
    except:
        print(row['RPID'])

C:\Users\jacks\AppData\Local\Temp\ipykernel_13532\3956106220.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ds = pd.Series()
